# Agenda, day 2

1. Recap and Q&A
2. Magic methods / "dunder" methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? It's easiest if we can use a data structure that is appropriate for solving that problem. Objects allow us to create a data structure that is specifically designed to solve a problem. Even though we might be using ints, strings, lists, and dicts to implement our object, the fact that we can think about it at a higher level makes it easier to work with.

The whole idea of object-oriented programming is thus: Create new data structures, along with methods, that together make it easier to think about and use these structures and thus solve our problems.

1. Methods are defined as part of the class body. And actually, methods are attributes on the class. However, we invoke them via the instance, and Python does a little magical substitution, turning the instance into the first argument passed to the method.
2. The first parameter in every method is traditionally called `self`, and gets that instance assigned to it.
3. If you try to invoke a method that doesn't exist, you'll get an "attribute error," with Python telling you that the method doesn't exist.
4. We define new "classes," factories for our data structures. We start the class definition with the reserved word `class`, then the name we want to give it, and then any number of methods that we might want to invoke on our data.
5. If we have a `Person` class, then it creates *instances* of `Person`. Every object has a `type`, and we can invoke `type()` on it to find out its class. In this way, `class` and `type` are two ways to say the same thing.
6. The most important method in a class is `__init__`, the initializer. (We pronounce it "dunder init," because it has a double underscore before and after the word "init.") Its job is to add attributes to a new object after that object is created, but before it's returned to the caller. Typically, we'll assign many attributes based on parameters, but we can add fewer or more, depending on what we want to do. In theory, you can add attributes to an object whenever you want... but it's a good idea to only add new attributes in `__init__`, for easier maintenance.
7. Each time we invoke a method on an instance, the first argument is the instance itself, and is assigned to `self`. You can use another word instead of `self`, and if you forget to make `self` the first parameter, then whatever is the first parameter will get the instance assigned to it. But you really should use `self`.
8. Python doesn't have protected or private status for attributes. (Other languages do, and they call attributes "instance variables" when they're set on `self`.) Whereas other languages have getters and setters so that people don't access those values directly, Python *encourages* us to use them directly!
   

In [1]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0   # attribute that I set, with a default/starter value, without any connection to a parameter

    def greet(self):
        return f'Hello, {self.name}!'  # if I just say "name", and not "self.name", then Python will not find my attribute

p = Person('Reuven', 46)    

In [2]:
p.greet()   # the same as Person.greet(p)

'Hello, Reuven!'

In [3]:
Person.greet(p)

'Hello, Reuven!'

In [4]:
p.name

'Reuven'

In [5]:
p.name = 'whatever'

In [6]:
p.name

'whatever'

In [7]:
p.greet()

'Hello, whatever!'

In [8]:
p.bank_balance

0

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on an instance of `Cellphone. It'll return a string saying, "Calling" and then print the number that is calling. You'll have to provide an argument, the number to call.

In [14]:
class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model

    def call(self, other_number):
        return f'Phone {self.number} is calling {other_number}'

c1 = Cellphone(12345, 'iPhone')
c2 = Cellphone(67890, 'Android')

c1.call(2468)

'Phone 12345 is calling 2468'

In [15]:
c2.call(c1.number)  

'Phone 67890 is calling 12345'

In [10]:
type(c1)

__main__.Cellphone

In [11]:
vars(c1)

{'number': 12345, 'model': 'iPhone'}

In [12]:
type(c2)

__main__.Cellphone

In [13]:
vars(c2)

{'number': 67890, 'model': 'Android'}

In [16]:
# what happens if I print my cellphones?

print(c1)

In [17]:
print(c2)

# Magic methods

When we perform certain operations in Python, behind the scenes, the operator is translated into a method call. For example, when we compare values with `==`, there's really a method that is being invoked.

Every operator that you can think of, plus many you probably cannot, are all implemented using methods.

The thing is, these methods don't always need to be defined. And you probably shoudn't be invoking them on your own very often. Plus, if we define these methods, then we really change the way that our objects work, because Python sees the method and uses it .

For all of these reasons, these special methods, known as "magic methods," have special names. All of their names are "dunders," meaning that they start and end with a double underscore, `__`. (That's not `_`, but rather `__` both before and after the method name.)

I want to show you some magic methods now, and we'll use them in a number of classes.

It's typical to implement a number of these methods. Also, new Python functionality is generally implemented by adding new dunder methods.

You should *not* name a method with dunders unless you know what you're doing! You might get yourself into trouble, either now or down the road if/when Python uses that name.

# Simple magic method: `__len__`

When you call `len` (the function) on an object, it in turn looks for the `__len__` method on that object.



In [18]:
len('abcd')

4

In [19]:
len(1234)

TypeError: object of type 'int' has no len()

In [20]:
class Person:
    def __init__(self, name):
        self.name = name

p = Person('Reuven')

len(p)

TypeError: object of type 'Person' has no len()

In [21]:
# I'm going to add a __len__ method that returns the length of the name

class Person:
    def __init__(self, name):
        self.name = name
    def __len__(self):
        return len(self.name)

p = Person('Reuven')

len(p)

6

# What happened?

1. I invoked `len(p)`
2. This was translated into `p.__len__()` by Python
3. Because `__len__` is defined on `Person`, it ran and its return value was returned to the caller.

# Exercise: How many scoops?

1. Yesterday, we defined both `Scoop` and `Bowl`. Any instance of `Bowl` has a `scoops` attribute, a list of `Scoop` instances.
2. Add the capability to invoke `len` on an instance of `Bowl`. That'll return the number of scoops in the bowl.
3. After doing so, test that it works -- call `len` on a `Bowl`, then add a new `Scoop`, then measure again.

In [24]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()
print(list_of_flavors)     # print forwards

print(len(b))

['chocolate', 'vanilla', 'coffee']
3


In [25]:
s = 'abcde'

In [26]:
s.__len__()  # don't do this, but *can* you?

5

# Warning:

Make sure that you spell the magic methods correctly!

I've often seen people accidentally write `__int__` instead of `__init__`. That is the magic method that is invoked when you want to turn a value into an integer, when you invoke `int()` on it! 

# Next up:

1. Methods for `__str__` and `__repr__`
2. A handful of other magic methods, as well

# Printing our objects

If we try to print our cellphone or scoop or bowl objects, we find that they are super ugly.

In [27]:
print(p)  # Person

In [28]:
print(c1)

In [29]:
print(c2)

What's happening here?

We can actually use `print` on any object in Python. That's because `print` invokes `str` on whatever it's going to print. And everything in Python knows how to turn itself into a string.

How does the object know how to turn itself into a string? Because there is a default implementation of the `__str__` method that is shared by all objects. If we do nothing, then every object that doesn't have a special way to display itself will use this ugly way.

But this also explains what we can do to change things: If we implement `__str__` on our object, then our method will be invoked instead of the default, and then we can customize what string is returned. Note that `__str__` *must* return a string! But the string you do return can be anything at all.

In [33]:
class Person:
    def __init__(self, name):
        self.name = name
    def __len__(self):
        return len(self.name)
    def __str__(self):
        return f'Person with name = {self.name}!'

p = Person('Reuven')
print(p)  # -> print(str(p)) -> print(p.__str__())

Person with name = Reuven!


In [34]:
str(p)  # invoke str on p, what do I get back?

'Person with name = Reuven!'

# Exercise: Printable scoops

1. Modify the `Scoop` class such that printing a scoop gives us a string, "Scoop of FLAVOR".
2. Make sure that this works by printing a number of scoops.


In [36]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()
print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee


# Exercise: Printable *bowls*

1. Now I want you to make it possible to say `print(b)`, where `b` is an instance of `Bowl`. Doing so should display "Bowl with:" and then each of the scoops on a line by itself.
2. (If you can number the scoops, even better, but don't worry about it too much.)
3. The printout for each of the scoops can be the result of invoking `str(a_scoop)`.

In [45]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

    def __str__(self):
        output = 'Bowl of: \n'

        # for index, one_scoop in enumerate(self.scoops, 1):
        #     output += f'\t{index}: {one_scoop}\n'
        
        return output + '\n'.join([f'\t{index}: {one_scoop}'
                   for index, one_scoop in enumerate(self.scoops, 1)])

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()

print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

print('*** Now printing the bowl ***')
print(b)  

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
*** Now printing the bowl ***
Bowl of: 
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee


In [46]:
# remember that we can look directly at b.scoops!
# what happens now that we have implemented __str__ on Scoop? 

b.scoops

In [47]:
print(b.scoops)

[<__main__.Scoop object at 0x108ad4ec0>, <__main__.Scoop object at 0x10fdfefd0>, <__main__.Scoop object at 0x10fdff4d0>]


# `__str__` and `__repr__`

It turns out that there are *two* ways to turn a Python object into a string:

- `__str__` is the method used when we invoke `print` or `str` on an object. `__str__` is supposed to return a string that is for public consumption, for the end user to view.
- `__repr__` is the method used when an object should be turned into a string for programmers, debuggers, and other people maintaining the code who want to see more of the internals.

When we defined `__str__` on `Scoop`, we still left the default of `__repr__`, which is used in Jupyter and debuggers when we print a list, and the list contains `Scoop` objects.

We could:

1. Define `__repr__` as well, perhaps even to be the same as `__str__`.
2. We could not define `__str__`, and instead define `__repr__`. In such a case, then `__repr__` is used for both itself and `__str__`.

Officially, 

- `__str__` is for end users.
- `__repr__` is for programmers.

If you don't define `__str__` but do define `__repr__`, then it handles both cases. I normally do that; I don't define `__str__` but I do define `__repr__`. If and when I want end users to see something specific to them, I add `__str__` later on, which doesn't have the "internal" information.

In [48]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl of: \n'

        # for index, one_scoop in enumerate(self.scoops, 1):
        #     output += f'\t{index}: {one_scoop}\n'
        
        return output + '\n'.join([f'\t{index}: {one_scoop}'
                   for index, one_scoop in enumerate(self.scoops, 1)])

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()

print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

print('*** Now printing the bowl ***')
print(b)  

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
*** Now printing the bowl ***
Bowl of: 
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee


In [49]:
b.scoops

[Scoop of chocolate, Scoop of vanilla, Scoop of coffee]

# Other magic methods

Magic methods are *everywhere* in Python! Let's talk about a few of them.

- `__eq__` -- this method is invoked whenever you use `==`. The item on the left of the operator is where the method is invoked, and the value on the right is passed as an additional argument.

In [50]:
class Person:
    def __init__(self, name):
        self.name = name

p1 = Person('abcd')
p2 = Person('abcd')

p1 == p2   # are these equal?

False

In [51]:
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):    # if I say a == b, it's translated into a.__eq__(b)
        return self.name == other.name

p1 = Person('abcd')
p2 = Person('abcd')

p1 == p2   # are these equal?

True

In [52]:
p1 == 5

AttributeError: 'int' object has no attribute 'name'

In [53]:
# one standard way to handle things: Check if it has the attribute!

class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):   
        if hasattr(other, 'name'):   # if the "other" object has the "name" attribute..
            return self.name == other.name

        return False  # if not, then they are not equal

p1 = Person('abcd')
p2 = Person('abcd')

p1 == p2   # are these equal?

True

In [54]:
p1 == 5

False

# Magic method `__getitem__`

Have you ever noticed that:

- We retrieve from a string with `[]`
- We retrieve from a list with `[]`
- We retrieve from a tuple with `[]`
- We retrieve from a dict with `[]'

Why? And how?

All four of these classes implement the `__getitem__` method. When we invoke `a[b]`, that invokes `a.__getitem__(b)`. What the class does with that method call depends on what it's supposed to do.

In [55]:

class Person:
    def __init__(self, name):
        self.name = name

    def __getitem__(self, index):
        return self.name[index]

p = Person('abcdefghij')
p[3]

'd'

# Exercise: Retrieve one scoop

1. Make it possible to retrieve a `Scoop` from an instance of `Bowl` by specifying the flavor.
2. That is, if I say `b['chocolate']`, then it should either return the first scoop with `'chocolate'` as the flavor, or return `None` to indicate that it didn't find that.

In [60]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl of: \n'

        # for index, one_scoop in enumerate(self.scoops, 1):
        #     output += f'\t{index}: {one_scoop}\n'
        
        return output + '\n'.join([f'\t{index}: {one_scoop}'
                   for index, one_scoop in enumerate(self.scoops, 1)])

    def __getitem__(self, flavor):
        for one_scoop in self.scoops:
            if one_scoop.flavor == flavor:
                return one_scoop

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()

print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

print('*** Now printing the bowl ***')
print(b)  

# let's retrieve a scoop by its numeric index
print(b['vanilla'])  

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
*** Now printing the bowl ***
Bowl of: 
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee
Scoop of vanilla


# Next up

1. Class attributes
2. ICPO -- searching for attributes
3. Inheritance

# How should we order magic methods?

- `__init__` should always come first
- I would suggest (generally) having all magic methods before others

I really don't think there's a convention on this.

# Class attributes

We have already said that everything in Python is an object. We're now going to see that *classes* are objects, too.

We've also said that every object in Python has attributes. We're now going to see that *classes* have attributes, too.

Yes, at the end, we'll see that instances have attributes and classes have attributes.

There is no such thing in Python as an attribute that is "shared" across objects. The idea that an attribute will exist jointly on a class and its instances does not exist.

In [61]:
# Let's say that we have a Person class

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [62]:
# My boss comes to me and says: Our customers love the Person class!
# They want some new functionality, to know how many instances have been created

# answer #1: a global variable, population!

population = 0

class Person:
    def __init__(self, name):
        self.name = name
        population += 1    # add 1 to the global variable population!

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0


UnboundLocalError: cannot access local variable 'population' where it is not associated with a value

When it comes to global variables in Python, either the variable exists and has a value or it doesn't exist. There is no such thing as a variable existing but being "unbound" to any value.

However, functions are different! When we write our function, Python notices the names of the variables to which we'll be assigning. It marks those as local variables. When we invoke our function, and we assign to a local variable, all is good. But if we try to retrieve from a local variable before we have assigned to it... we get UnboundLocalError.

In this case, we assigned to `population` inside of the function. Python assumes that we want to create a new local variable with the same name as the global one. Then, we use `+=` on it. That requires retrieving the current value of the variable... but it has no value! We get the exception.

We can get around this problem by declaring the `population` variable to be `global`. That tells Python's compiler not to mark it as local, and we'll thus assign where we want.

In [63]:
# My boss comes to me and says: Our customers love the Person class!
# They want some new functionality, to know how many instances have been created

# answer #1: a global variable, population!

population = 0

class Person:
    def __init__(self, name):
        global population
        self.name = name
        population += 1    # add 1 to the global variable population!

    def greet(self):
        return f'Hello, {self.name}!'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


In [65]:
# it's really bad for a function to use "global" so that it can assign to a global variable.
# sometimes there isn't any choice... but maybe we have one here!

# option 2: Define a class attribute

class Person:
    def __init__(self, name):
        self.name = name
        Person.population += 1    # add 1 to the attribute Person.population

    def greet(self):
        return f'Hello, {self.name}!'

# after the class as been defined, I'll add a new attribute to it
Person.population = 0

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


In [66]:
# option 3: Define a class attribute inside of the class definition!
# inside of the class body, there is an implicit understanding that all assignment
#  is to class attributes, not to variables

class Person:
    population = 0    # actually Person.population

    def __init__(self, name):  # actually Person.__init__
        self.name = name
        Person.population += 1    # add 1 to the attribute Person.population

    def greet(self):  # Person.greet
        return f'Hello, {self.name}!'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1!
Hello, name2!


# When do we want to use class attributes?

1. If there's a resource that we want to share among our instances. We can say `Person.population` from any of our instances, and it'll work fine.
2. To set constants, or generally values that will be useful. The class is a natural place to put such definitions.
3. Methods are class attributes as well, but that's a bit different.

# Exercise: Limit scoops

1. Modify the `Bowl` class, such that at most three scoops can be added into the bowl. If we ever get a request to add another scoop to a full bowl, we ignore the requests.
2. It shouldn't matter if we invoke `add_scoop` or `add_scoops`, or how many times we invoke them, or how many arguments we pass to `add_scoops`.

In [69]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')
    
class Bowl:
    MAX_SCOOPS = 3
    
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        if len(self.scoops) >= Bowl.MAX_SCOOPS:
            return
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:   
            self.add_scoop(one_scoop)  

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl of: \n'
        return output + '\n'.join([f'\t{index}: {one_scoop}'
                   for index, one_scoop in enumerate(self.scoops, 1)])

    def __getitem__(self, flavor):
        for one_scoop in self.scoops:
            if one_scoop.flavor == flavor:
                return one_scoop

b = Bowl()   
b.add_scoops(s1, s2, s3)  
b.add_scoops(s4, s5)  
b.add_scoops(s6)

print(b)

Bowl of: 
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee


In [70]:
# what if I do this?

class Person:
    population = 0    # actually Person.population

    def __init__(self, name):  # actually Person.__init__
        self.name = name
        Person.population += 1    # add 1 to the attribute Person.population

    def greet(self):  # Person.greet
        return f'Hello, {self.name}!'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')
print(f'After, p1.population = {p1.population}')
print(f'After, p2.population = {p2.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
After, p1.population = 2
After, p2.population = 2
Hello, name1!
Hello, name2!


# Attribute lookup

When we ask Python to retrieve the value of an attribute on an object, we think that we'll either get a value back or that we'll get an exception, an `AttributeError`, indicating that the attribute doesn't exist.

But Python doesn't work like this. If we ask for an attribute from object, Python tries in a few places before giving up:

- On the instance itself that requested
- On the class of the instance

In our Person example, when we asked for `p1.population`, this is what happened:

- Python asked `p1` if it has an attribute named `population`. The answer was "no."
- Python then asked `p1`'s class, `Person`, if it has an attribute named `population`, the answer was "yes," and we got the number back.

You have already seen this before, because you have been executing methods. Remember that when we invoke

    p1.greet()

Python:

- Turns to `p1` and asks: Do you have an attribute named `greet`? The answer is "no."
- Turns to `p1`'s class and asks: Do you have an attribute named `greet`? The answer is "yes." That method is retrieved and invoked.

# My term for this

ICPO:

- First, we look on the *instance* that was named in the Python expression
- Then, we look on the *class* of that instance
- Then we look on that class's *parents* (via inheritance)
- Finally, we look at `object`, the ultimate parent in the Python class hierarchy.

In [71]:
# it's tempting to do this:

class Person:
    population = 0   

    def __init__(self, name):  # actually Person.__init__
        self.name = name
        self.population += 1  

    def greet(self): 
        return f'Hello, {self.name}!'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')
print(f'After, p1.population = {p1.population}')
print(f'After, p2.population = {p2.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 0
After, p1.population = 1
After, p2.population = 1
Hello, name1!
Hello, name2!


In the above code, every time we invoked `__init__`:

- We ran `self.population += 1`
- This first retrieved the current value of `self.population`.
- On a new object, there is no attribute `self.population`. So we were told to check with the class, `Person`.
- `Person` has an attribute `population`, with a value of 0. We grab that, add 1, and then assign to `self.population`, creating a new attribute on the instance that has nothing to do with the attribute of the same name on the class.

# What's my point?

- It's OK to retrieve a class attribute either via the instance or via the class.
- If you're setting a class attribute, you *must* assign via the class. Assigning via the instance will create this kind of chaos, with a new attribute on the instance that we really wanted on the class.


In [72]:
type(str)

type

In [73]:
type(Bowl)

type

In [74]:
type(Scoop)

type

In [75]:
# if type is the class for every class
# then what is the type of type?

type(type)  

type

# Next up

1. Inheritance
2. Finish up all of the ICPO levels (parent and `object`)
3. How to use inheritance

In [76]:
# Once again, we have a Person class that our company has developed.
# Our clients are super happy with it!

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())  # does p1 have greet? No. Does Person have greet? Yes! We run that.
print(p2.greet())

Hello, name1
Hello, name2


In [77]:
# Our customers are so happy with our software, they want to use it to manage
# more of their business. That means having not just a Person class, but also
# an Employee class. Employees are identical to People, except that they have one
# additional piece of information associated with them -- their ID number.

# Option 1: Copy and paste!

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet()) 
print(p2.greet())

class Employee:
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

    def greet(self):
        return f'Hello, {self.name}'

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet()) 
print(e2.greet())

Hello, name1
Hello, name2
Hello, emp1
Hello, emp2


In [ ]:
# My boss says: You have two classes here, one of which is nearly identical to the other.
# Couldn't you use inheritance?

# Option 2: Inheritance

# Inheritance

Many *many* classes in object-oriented programming seem to stress inheritance as one of the most important parts of this paradigm. It is important, and useful... but not everywhere.

The whole point of inheritance is to avoid writing duplicate code. If there's already a class that basically, mostly does what you want in your new class, then you *inherit* from the existing one and just tweak/modify what needs changing.

This is true if you have written the original class, or a colleague has, or a third party has. By inheriting from that other class, you can then make a new class that's more specific than that one, and closer to your needs.

If you have two classes, one of which is a specific version of the other, then inheritance seems like a good idea. 

Inheritance works via attribute lookup:

- If an attribute is on an instance, we get that value.
- If not, then we look at the instance's class.
- If not there, then we look at the *PARENT* of that class. In other words, we can say that one class *inherits* from another.
- In fact, we can have several levels of parents
- Eventually, we get to the final, ultimate, highest parent, known as `object`.
- If the attribute (method) we want isn't found anywhere, then we get an error.

In [78]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet()) 
print(p2.greet())

class Employee(Person):    # putting a class in () means: Employee inherits from Person, aka Employee is-a Person
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

    def greet(self):
        return f'Hello, {self.name}'

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet()) 
print(e2.greet())

Hello, name1
Hello, name2
Hello, emp1
Hello, emp2


In [80]:
# because "greet" is identical in Employee (the child class) and Person (the parent class), we can
# just get rid of Employee.greet.

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet()) 
print(p2.greet())

class Employee(Person):    # putting a class in () means: Employee inherits from Person, aka Employee is-a Person
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet()) # does e1 have greet? No. Does Employee have greet? No. Does Person (Employee's parent) have greet? Yes.
print(e2.greet())

Hello, name1
Hello, name2
Hello, emp1
Hello, emp2


In [ ]:
# what about our assignment to self.name in both Person.__init__ and Employee.__init__?
# what if we remove the assignment from Employee? Can we rely on Person.__init__ to set it?

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet()) 
print(p2.greet())

class Employee(Person):    # putting a class in () means: Employee inherits from Person, aka Employee is-a Person
    def __init__(self, name, id_number):
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet()) # does e1 have greet? No. Does Employee have greet? No. Does Person (Employee's parent) have greet? Yes.
print(e2.greet())